In [1]:
%load_ext autoreload
%autoreload 2

# BTM用に､10000分割したテキストをre-clusteringするための分配処理

In [2]:
#loaderの設定などの練習
import json
from loaders.CommonCrawlDataset import CommonCrawlDataset

In [3]:
#10000分割されたjsonlファイルのパスを取得

import glob
import random

jsonl_path_list=glob.glob("../data/dedup_categorized/*/*.jsonl")
random.shuffle(jsonl_path_list)
cid_to_path_list={}
for jsonl_path in jsonl_path_list:
    cid=jsonl_path.split("/")[-2]
    if cid not in cid_to_path_list:
        cid_to_path_list[cid]=[]
    cid_to_path_list[cid].append(jsonl_path)

In [4]:
from tqdm import tqdm
#代表的な*件を取得
max_texts=10
cid_to_texts={}
for cid, path_list in tqdm(cid_to_path_list.items()):
    texts=[]
    for path in path_list:
        cnt=0
        with open(path) as f:
            for i, line in enumerate(f):
                t=json.loads(line)["text"]
                if len(t)<10:
                    continue
                texts.append(t)
                cnt+=1
                if cnt>max_texts:
                    break
    cid_to_texts[cid]=texts

  7%|▋         | 255/3815 [00:00<00:01, 2549.40it/s]

100%|██████████| 3815/3815 [00:00<00:00, 4481.23it/s]


In [5]:
import sys
import numpy as np
sys.path.append("../01web_codes")
from gensim.models.fasttext import load_facebook_model
from tqdm import tqdm
from src.classify.Text2Vec import Text2Vec

t2v = Text2Vec(load_facebook_model('../data/model/cc.ja.300.bin'))

In [6]:
#各カテゴリのテキストをベクトル化
cid_to_vecs={}
check_length=100
for cid, texts in tqdm(cid_to_texts.items()):
    vecs = [t2v.text2vec(i[:check_length]) for i in texts]
    vecs=np.array(vecs)
    vecs=np.mean(vecs, axis=0)
    cid_to_vecs[cid]=vecs



100%|██████████| 3815/3815 [00:05<00:00, 749.72it/s] 


In [7]:
vec_array=list(cid_to_vecs.values())
vec_array=[i for i in vec_array if i.shape==(300,)]
vecs.shape

(300,)

In [17]:
n_clusters=5
from sklearn.cluster import MiniBatchKMeans,KMeans
vec_array=np.array(vec_array).astype(np.double)
# k-meansクラスタリング
print("clustering...")
kmeans = KMeans(n_clusters=n_clusters, random_state=1).fit(vec_array)

clustering...


In [18]:
cid_to_label={}
for cid, vec in cid_to_vecs.items():
    try:
        label=kmeans.predict([vec])[0]
        cid_to_label[cid]=label
    except Exception as e:
        print(cid,e)

In [19]:
label_to_path_list={}
for cid, label in cid_to_label.items():
    if str(label) not in label_to_path_list:
        label_to_path_list[str(label)]=[]
    path=f"../data/dedup_categorized/{cid}/*.jsonl"
    label_to_path_list[str(label)].append(path)


In [20]:
import json
with open("../data/clustered_path.json", "w") as f:
    json.dump(label_to_path_list, f, indent=4)

In [37]:
#行数を数える
import os
import subprocess
import json
with open("../data/clustered_path.json") as f:
    label_to_path_list = json.load(f)


from tqdm import tqdm

label_to_article_count = {}

for label, dir_paths in tqdm(label_to_path_list.items()):
    label_to_article_count[label] = 0
    for dir_path in tqdm(dir_paths):
        dir_path = dir_path.split("/")[:-1]
        dir_path = "/".join(dir_path)
        # 対象のディレクトリパス
        #dir_path = '/media/hatakeyama/python/Dataset_for_BTM/data/dedup_categorized/5538'

        # .jsonlファイルの絶対パスを取得
        file_paths = [os.path.join(root, file)
                    for root, _, files in os.walk(dir_path)
                    for file in files if file.endswith('.jsonl')]

        # ファイルが見つかった場合にのみwcを実行
        if file_paths:
            # wc -lコマンドをファイルリストに対して実行
            result = subprocess.run(['wc', '-l'] + file_paths, capture_output=True, text=True)
            count=result.stdout.split('\n')[-2].strip().split(' ')[0]
            label_to_article_count[label] += int(count)-len(file_paths) #最終行は空行なので除外
        

100%|██████████| 5/5 [00:09<00:00,  1.88s/it]


In [38]:
label_to_article_count

{'4': 9755, '3': 3862, '1': 3983, '0': 2757, '2': 5749}

In [34]:

with open("../data/clustered_n.json", "w") as f:
    json.dump(label_to_article_count, f, indent=4)

In [35]:
n_clusters=5
from loaders.CommonCrawlDataset import CommonCrawlDataset
for i in range(n_clusters):
    print("--------\ncluster: ",i)
    cc1_loader=CommonCrawlDataset(label_to_path_list[str(i)])
    cc1_iter=iter(cc1_loader)
    for i in range(10):
        print(next(cc1_iter))

--------
cluster:  0
{'text': '(レンタ)のレンタル制度は最大48時間レンタル可能というのが特徴的です。【ハーレクインコミック】見せかけの恋人テーマセットvol.2(岡本慶子:一重夕子:岸本景子:天野なすの)はじめ作品数は5万札以上と品ぞろえは豊富と言えます。\n(レンタ)利用するなら【ハーレクインコミック】見せかけの恋人テーマセットvol.2(岡本慶子:一重夕子:岸本景子:天野なすの)無料お試しや100円以下の作品を読みたい人、もしくはレンタルして気に入ったら購入しようか考えている人が利用するにはおすすめと言えます。\n今日ではスマホを通してエンジョイできるアプリも、グングン多様化していると言えます。無料アニメ動画を筆頭に、無料電子書籍「Renta!(レンタ)」【ハーレクインコミック】見せかけの恋人テーマセットvol.2や無料で遊べるスマホゲームなど、いろんなコンテンツの中から選ぶことが可能です。\n今までは単行本という形になったものを買っていたという方も、電子書籍「Renta!(レンタ)」【ハーレクインコミック】見せかけの恋人テーマセットvol.2を利用した途端に、普通の本には戻れないということがほとんどらしいです。コミックサイトも1つや2つではありませんから、絶対に比較してから選ぶことが大切です。\nチンクロボーイズ♂アソコで魅せる狂演乱舞(ふじい葛西)のレンタルサイト電子貸本Renta!(レンタ)のレンタル制度は最大48時間レンタル可能というのが特徴的です。チンクロボーイズ♂アソコで魅せる狂演乱舞(ふじい葛西)はじめ作品数は5万札以上と品ぞろえは豊富と言えます。\n漫画が趣味な人からすれば、コレクションのための単行本は大事だと思いますが、「読めればそれでOK」という人だとすれば、スマホで見れる電子書籍「Renta!(レンタ)」チンクロボーイズ♂アソコで魅せる狂演乱舞「Renta!(レンタ)」の方が断然便利だと言えます。'}
{'text': '今、チケットの取れない講談師として大注目されている神田松之丞を1冊まるごとの大特集した「Pen+(ペン・プラス)『完全保存版1冊まるごと、神田松之丞』(メディアハウスムック)」がCCCメディアハウスから2018年10月25日に発売された。\n神田松之丞は2018年現在、まだ真打ではなく、二

In [36]:
#モデル保存
import pickle
with open("../data/second_kmeans_model.pkl", "wb") as f:
    pickle.dump(kmeans, f)